In [12]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import calendar
import time
from scipy.stats import binned_statistic_2d
import glob

In [13]:
# column_names = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude',
#                 'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'payment_type',
#                  'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

In [14]:
regions = pd.read_csv('./TAXI DATA/regions.csv', sep=';')
binx = np.sort(pd.concat([regions['south'],regions['north']]).unique())
biny = np.sort(pd.concat([regions['west'],regions['east']]).unique())

Вспомогательные функции агрегации

In [15]:
def datetime_to_int(dt):
    '''
    Функция преобразует дату к числовому виду yyyymmddhh для дальнейшего использования при агрегации 
    '''
    return 1e6*dt.year + 1e4*dt.month + 1e2*dt.day+dt.hour

In [16]:
def get_time_range(year,month,adddays,addhours):
    '''
    Функция формирует перечень дат в числовом виде yyyymmddhh за период с первое по последнее число 
    месяца month в году year с шагом 1 час
    '''
    def perdelta(start, end, delta):
        curr = start
        while curr < end:
            yield curr
            curr += delta
    last_day= datetime(year,month,calendar.monthrange(year,month)[1])+timedelta(days=adddays,hours=addhours)
    datetime_range=[]  
    for result in perdelta(datetime(year, month, 1), last_day, timedelta(hours=1)):
        datetime_range.append(datetime_to_int(result))
    return datetime_range    

Подготовка данных

In [17]:
def altered_data(data):
    data.columns = column_names
    
    # фильтрация значений
    filter1 = data['dropoff_datetime'] > data['pickup_datetime']
    filter2 = data['passenger_count'] > 0
    filter3 = data['trip_distance'] > 0
    filter4 = (data['pickup_longitude'] >= -74.25559) & (data['pickup_longitude'] <= -73.70001)
    filter5 = (data['pickup_latitude'] >= 40.49612) & (data['pickup_latitude'] <= 40.91553)
    filtered_data = data[filter1 & filter2 & filter3 & filter4 & filter5]
    
    # преобразование столбцов
    filtered_data.pickup_datetime = pd.to_datetime(filtered_data.pickup_datetime)
    filtered_data.dropoff_datetime = pd.to_datetime(filtered_data.dropoff_datetime)
    
    # добавление признаков
    agg_id_zones = binned_statistic_2d(x = filtered_data['pickup_latitude'].values, 
                                       y = filtered_data['pickup_longitude'].values, 
                                       values = None, statistic = 'count', 
                                       bins = [binx, biny], expand_binnumbers = True)
    filtered_data['id_zone'] = 50*(agg_id_zones.binnumber[1, :] - 1) + agg_id_zones.binnumber[0, :]
    filtered_data['int_pickup_data'] = filtered_data.pickup_datetime.apply(datetime_to_int)
    
    return filtered_data

Агрегирование данных

In [18]:
def get_agr_data(data, year, month):
    binx = get_time_range(int(year),int(month),1,1)
    biny = np.arange(0, 2501)
    res = binned_statistic_2d(data.int_pickup_data, data.id_zone,
                                   None, 'count',
                                   bins = [binx, biny])
    df = pd.DataFrame(res.statistic, index = get_time_range(int(year),int(month),1,0), columns = np.arange(1, 2501))
    df.index = df.index.map(int)
    df.index=df.index.map(lambda x:datetime.strptime(str(x), "%Y%m%d%H"))
    df.index.names = ['dt']
    return df

In [19]:
def get_special_agr_data(data, year, month, feature_name):
    binx = get_time_range(int(year),int(month),1,1)
    biny = np.arange(0, 2501)
    res = binned_statistic_2d(data.int_pickup_data, data.id_zone,
                                   data[feature_name], 'mean',
                                   bins = [binx, biny])
    df = pd.DataFrame(np.nan_to_num(res.statistic), index = get_time_range(int(year),int(month),1,0), 
                      columns = np.arange(1, 2501))
    df.index = df.index.map(int)
    df.index=df.index.map(lambda x:datetime.strptime(str(x), "%Y%m%d%H"))
    df.index.names = ['dt']
    return df

Сборка общего датафрейма

In [20]:
year_list = ['2014']
month_list = ['01','02','03','04','05','06', '07', '08', '09', '10', '11', '12']

Проведем агрегирование основных данных - плотности поездок по времени и id зон. Это то, что делалось на первой неделе.

In [21]:
column_names = ['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'pickup_longitude',
                'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude']
# 'payment_type',
#                  'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

In [24]:
def calculate_df(year_list, month_list,):
    df = pd.DataFrame()
    for year in year_list:
        print year
        for month in month_list:
            print month,
            data = pd.read_csv('./TAXI DATA/'+year+'/yellow_tripdata_'+year+'-'+month+'.csv')
            data = data.iloc[:, :11]
            alt_data = altered_data(data)
            agr_data = get_agr_data(alt_data, year, month)
            df = pd.concat([df, agr_data])
    return df                    

In [25]:
full_df = calculate_df(year_list, month_list)
print full_df.shape
full_df.to_csv('./TAXI DATA/CSVs/agr_2014.csv')

2014
01

C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\scipy\stats\_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]
C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

 02

C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\IPython\core\interactiveshell.py:2818: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


 03 04 05 06 07 08 09 10 11 12 (8760, 2500)


Далее вычленим новые признаки из сырых данных:
* Средняя протяженность поездок
* Среднее число пассажиров
* Средняя стоимость поездок

In [12]:
def write_special_df(year_list, month_list, feature_name):
    df = pd.DataFrame()
    for year in year_list:
        print year
        for month in month_list:
            print month,
            data = pd.read_csv('./TAXI DATA/'+year+'/yellow_tripdata_'+year+'-'+month+'.csv')
            alt_data = altered_data(data)
            agr_data = get_special_agr_data(alt_data, year, month, feature_name)
            df = pd.concat([df, agr_data])
    df.to_csv('./TAXI DATA/CSVs/agr_2016_mean_'+str(feature_name)+'.csv')
    print 

In [13]:
feature_list_names = ['trip_distance', 'passenger_count', 'total_amount']

In [16]:
for feature_name in feature_list_names:
    print feature_name
    write_special_df(year_list, month_list, feature_name)

trip_distance
2016
01

C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\GusarovPC\AppData\Local\conda\conda\envs\taxi3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


 02 03 04 05 06
passenger_count
2016
01 02 03 04 05 06
total_amount
2016
01 02 03 04 05 06
